# Sentiment Analysis

In [1]:
!pip install pysastrawi
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
import re

from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from gensim.models import LdaModel
from gensim.corpora import Dictionary

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
review = {'review':['Aplikasinya suka eror dan terlogout secara tiba-tiba',
          'Saya sangat khawatir dengan aplikasi Danamon Mobile Banking. Meskipun berguna untuk melakukan transaksi perbankan secara mudah dan cepat, aplikasi ini memiliki sejumlah masalah yang sering kali mengganggu pengalaman pengguna. Beberapa fitur sering mengalami kesalahan teknis dan kadang-kadang sulit diakses. Selain itu, keamanan data juga menjadi kekhawatiran saya karena beberapa kali terdengar kabar tentang pelanggaran keamanan pada aplikasi ini. Saya berharap Danamon dapat segera memperbaiki masalah-masalah tersebut agar pengguna dapat merasa lebih aman dan nyaman saat menggunakan aplikasi ini.',
          'Saya sangat senang dengan pengalaman menggunakan aplikasi Danamon Mobile Banking. Aplikasi ini sangat mudah digunakan dan memberikan akses yang cepat dan aman ke rekening bank saya. Fitur-fitur yang disediakan, seperti transfer dana, pembayaran tagihan, dan cek saldo, semuanya berfungsi dengan baik. Selain itu, tampilan antarmuka yang intuitif membuat pengalaman pengguna menjadi lebih menyenangkan. Saya sangat merekomendasikan Danamon Mobile Banking kepada siapa saja yang mencari solusi perbankan digital terpercaya dan efisien.',
          'Saya telah menggunakan aplikasi Danamon Mobile Banking dan saya harus mengatakan bahwa pengalaman saya tidak begitu memuaskan. Meskipun aplikasi ini memiliki fitur dasar seperti cek saldo, transfer, dan pembayaran tagihan, namun antarmuka penggunaannya terasa kuno dan tidak intuitif. Saya juga sering mengalami masalah teknis seperti kesalahan saat melakukan transaksi atau lambatnya respons dari aplikasi ini. Selain itu, fitur keamanan yang disediakan juga terasa kurang memadai. Secara keseluruhan, saya merasa bahwa Danamon Mobile Banking perlu melakukan banyak perbaikan agar dapat bersaing dengan aplikasi perbankan digital lainnya di pasaran.',
          'Saya merasa frustasi dengan aplikasi mobile banking ini. Selalu ada masalah teknis yang menghambat transaksi saya. Selain itu, antarmukanya terkadang membingungkan dan sulit dinavigasi.',
          'Aplikasi mobile banking ini cukup standar. Saya dapat melakukan transaksi dasar seperti cek saldo dan transfer, tetapi tidak ada yang benar-benar menonjol. Perlu beberapa perbaikan untuk memberikan pengalaman yang lebih baik.']}


df = pd.DataFrame(review)

display(df)

,review
0,Aplikasinya suka eror dan terlogout secara tib...
1,Saya sangat khawatir dengan aplikasi Danamon M...
2,Saya sangat senang dengan pengalaman menggunak...
3,Saya telah menggunakan aplikasi Danamon Mobile...
4,Saya merasa frustasi dengan aplikasi mobile ba...
5,Aplikasi mobile banking ini cukup standar. Say...


In [4]:
pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
# Function to all review

def text_proses(teks):
  # lower text
  teks = teks.lower()

  # delete Mention
  teks = re.sub("@[A-Za-z0-9_]+", " ", teks)

  # delete Hashtag
  teks = re.sub("#[A-Za-z0-9_]+", " ", teks)

  # delete \n
  teks = re.sub(r"\\n", " ",teks)

  # delete Whitespace
  teks = teks.strip()

  # delete Link
  teks = re.sub(r"http\S+", " ", teks)
  teks = re.sub(r"www.\S+", " ", teks)

  # delete emoji etc
  teks = re.sub("[^A-Za-z\s']", " ", teks)

  return teks

In [6]:
# apply function using lambda x

df['new_text'] = df['review'].apply(lambda x: text_proses(x))
df

,review,new_text
0,Aplikasinya suka eror dan terlogout secara tib...,aplikasinya suka eror dan terlogout secara tib...
1,Saya sangat khawatir dengan aplikasi Danamon M...,saya sangat khawatir dengan aplikasi danamon m...
2,Saya sangat senang dengan pengalaman menggunak...,saya sangat senang dengan pengalaman menggunak...
3,Saya telah menggunakan aplikasi Danamon Mobile...,saya telah menggunakan aplikasi danamon mobile...
4,Saya merasa frustasi dengan aplikasi mobile ba...,saya merasa frustasi dengan aplikasi mobile ba...
5,Aplikasi mobile banking ini cukup standar. Say...,aplikasi mobile banking ini cukup standar say...


In [7]:
#change data types into string
df['new_text'] = df['new_text'].astype('str')

# apllying model to all review
df['result'] = df['new_text'].apply(lambda x:nlp(x))

# split sentiment and probabillity from results
df['sentiment'] =df['result'].apply(lambda x :(x[0]['label']))
df['proba'] = df['result'].apply(lambda x :(x[0]['score']))

In [8]:
df

,review,new_text,result,sentiment,proba
0,Aplikasinya suka eror dan terlogout secara tib...,aplikasinya suka eror dan terlogout secara tib...,"[{'label': 'negative', 'score': 0.998788297176...",negative,0.998788
1,Saya sangat khawatir dengan aplikasi Danamon M...,saya sangat khawatir dengan aplikasi danamon m...,"[{'label': 'negative', 'score': 0.982018589973...",negative,0.982019
2,Saya sangat senang dengan pengalaman menggunak...,saya sangat senang dengan pengalaman menggunak...,"[{'label': 'positive', 'score': 0.999044358730...",positive,0.999044
3,Saya telah menggunakan aplikasi Danamon Mobile...,saya telah menggunakan aplikasi danamon mobile...,"[{'label': 'negative', 'score': 0.996023654937...",negative,0.996024
4,Saya merasa frustasi dengan aplikasi mobile ba...,saya merasa frustasi dengan aplikasi mobile ba...,"[{'label': 'negative', 'score': 0.998575091361...",negative,0.998575
5,Aplikasi mobile banking ini cukup standar. Say...,aplikasi mobile banking ini cukup standar say...,"[{'label': 'negative', 'score': 0.502381682395...",negative,0.502382


# Categorization

In [9]:
df

,review,new_text,result,sentiment,proba
0,Aplikasinya suka eror dan terlogout secara tib...,aplikasinya suka eror dan terlogout secara tib...,"[{'label': 'negative', 'score': 0.998788297176...",negative,0.998788
1,Saya sangat khawatir dengan aplikasi Danamon M...,saya sangat khawatir dengan aplikasi danamon m...,"[{'label': 'negative', 'score': 0.982018589973...",negative,0.982019
2,Saya sangat senang dengan pengalaman menggunak...,saya sangat senang dengan pengalaman menggunak...,"[{'label': 'positive', 'score': 0.999044358730...",positive,0.999044
3,Saya telah menggunakan aplikasi Danamon Mobile...,saya telah menggunakan aplikasi danamon mobile...,"[{'label': 'negative', 'score': 0.996023654937...",negative,0.996024
4,Saya merasa frustasi dengan aplikasi mobile ba...,saya merasa frustasi dengan aplikasi mobile ba...,"[{'label': 'negative', 'score': 0.998575091361...",negative,0.998575
5,Aplikasi mobile banking ini cukup standar. Say...,aplikasi mobile banking ini cukup standar say...,"[{'label': 'negative', 'score': 0.502381682395...",negative,0.502382


In [10]:
# Stemming using sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stopwords using nltk
stpwds_id = list(set(stopwords.words('indonesian')))
additional_stopwords = ['ok', 'OK', 'ga', 'nya', 'yg']
stpwds_id.append(additional_stopwords)

In [11]:
# Membuuat Sebuah Function Agar Dapat Diterapkan pada DataFrame

def text_proses_inf(teks):
  # Mengubah Teks ke Lowercase
  teks = teks.lower()

  # Menghilangkan Mention
  teks = re.sub("@[A-Za-z0-9_]+", " ", teks)

  # Menghilangkan Hashtag
  teks = re.sub("#[A-Za-z0-9_]+", " ", teks)

  # Menghilangkan \n
  teks = re.sub(r"\\n", " ",teks)

  # Menghilangkan Whitespace
  teks = teks.strip()

  # Menghilangkan Link
  teks = re.sub(r"http\S+", " ", teks)
  teks = re.sub(r"www.\S+", " ", teks)

  # Menghilangkan yang Bukan Huruf seperti Emoji, Simbol Matematika (seperti μ), dst
  teks = re.sub("[^A-Za-z\s']", " ", teks)

  # Menghilangkan Underscores
  teks = teks.replace("_", " ")

  # Menghilangkan Karakter Alfabet Tunggal
  teks = re.sub(r"\b[a-zA-Z]\b", " ", teks)

  # Melakukan Tokenisasi
  tokens = word_tokenize(teks)

  # Menghilangkan Stopwords
  teks = ' '.join([word for word in tokens if word not in stpwds_id])

  # Melakukan Stemming
  teks = stemmer.stem(teks)

  return teks

In [12]:
# Applying Preprocessing to all Documents

df['text_processed'] = df['new_text'].apply(lambda x: text_proses_inf(x))
df

,review,new_text,result,sentiment,proba,text_processed
0,Aplikasinya suka eror dan terlogout secara tib...,aplikasinya suka eror dan terlogout secara tib...,"[{'label': 'negative', 'score': 0.998788297176...",negative,0.998788,aplikasi suka eror terlogout
1,Saya sangat khawatir dengan aplikasi Danamon M...,saya sangat khawatir dengan aplikasi danamon m...,"[{'label': 'negative', 'score': 0.982018589973...",negative,0.982019,khawatir aplikasi danamon mobile banking guna ...
2,Saya sangat senang dengan pengalaman menggunak...,saya sangat senang dengan pengalaman menggunak...,"[{'label': 'positive', 'score': 0.999044358730...",positive,0.999044,senang alam aplikasi danamon mobile banking ap...
3,Saya telah menggunakan aplikasi Danamon Mobile...,saya telah menggunakan aplikasi danamon mobile...,"[{'label': 'negative', 'score': 0.996023654937...",negative,0.996024,aplikasi danamon mobile banking alam muas apli...
4,Saya merasa frustasi dengan aplikasi mobile ba...,saya merasa frustasi dengan aplikasi mobile ba...,"[{'label': 'negative', 'score': 0.998575091361...",negative,0.998575,frustasi aplikasi mobile banking teknis hambat...
5,Aplikasi mobile banking ini cukup standar. Say...,aplikasi mobile banking ini cukup standar say...,"[{'label': 'negative', 'score': 0.502381682395...",negative,0.502382,aplikasi mobile banking standar transaksi dasa...


# Load Models

In [13]:
# Load the trained LDA model and dictionary
lda_model = LdaModel.load("/content/lda.model")
dictionary = Dictionary.load("/content/dictionary.dict")

In [14]:
topic_names = {0: 'User Experience',
               1: 'App Features',
               2: 'Questions and Engagement',
               3: 'Opinion on Banking App',
               4: 'Mixed Feedback and Technical Issues',
                }

In [15]:
# Function to perform inference and get the topic with the highest probability
def get_highest_probability_topic(lda_model, dictionary, new_document, topic_names):
    new_bow = dictionary.doc2bow(new_document.split())
    topic_distribution = lda_model.get_document_topics(new_bow, minimum_probability=0)

    highest_probability_topic = max(topic_distribution, key=lambda x: x[1])
    topic_id, probability = highest_probability_topic
    topic_name = topic_names.get(topic_id, f"Topic {topic_id}")

    return topic_name, probability

In [16]:
# Create an empty list to store the results
inferred_topics_list = []
inferred_probabilities_list = []

# Perform inference for each row in the DataFrame
for index, row in df.iterrows():
    new_document = row['text_processed']  

    inferred_topic, inferred_probability = get_highest_probability_topic(lda_model, dictionary, new_document, topic_names)
    inferred_topics_list.append(inferred_topic)
    inferred_probabilities_list.append(inferred_probability)

# Create new columns in the DataFrame to store the inferred topics and probabilities
df['inferred_topic'] = inferred_topics_list
df['inferred_probability'] = inferred_probabilities_list

# Save the updated DataFrame to a new CSV file
df.to_csv("inferred_results.csv", index=False)

# Print the first few rows of the updated DataFrame
df.head(6)

,review,new_text,result,sentiment,proba,text_processed,inferred_topic,inferred_probability
0,Aplikasinya suka eror dan terlogout secara tib...,aplikasinya suka eror dan terlogout secara tib...,"[{'label': 'negative', 'score': 0.998788297176...",negative,0.998788,aplikasi suka eror terlogout,Mixed Feedback and Technical Issues,0.426355
1,Saya sangat khawatir dengan aplikasi Danamon M...,saya sangat khawatir dengan aplikasi danamon m...,"[{'label': 'negative', 'score': 0.982018589973...",negative,0.982019,khawatir aplikasi danamon mobile banking guna ...,User Experience,0.386195
2,Saya sangat senang dengan pengalaman menggunak...,saya sangat senang dengan pengalaman menggunak...,"[{'label': 'positive', 'score': 0.999044358730...",positive,0.999044,senang alam aplikasi danamon mobile banking ap...,User Experience,0.450688
3,Saya telah menggunakan aplikasi Danamon Mobile...,saya telah menggunakan aplikasi danamon mobile...,"[{'label': 'negative', 'score': 0.996023654937...",negative,0.996024,aplikasi danamon mobile banking alam muas apli...,User Experience,0.445227
4,Saya merasa frustasi dengan aplikasi mobile ba...,saya merasa frustasi dengan aplikasi mobile ba...,"[{'label': 'negative', 'score': 0.998575091361...",negative,0.998575,frustasi aplikasi mobile banking teknis hambat...,Opinion on Banking App,0.896955
5,Aplikasi mobile banking ini cukup standar. Say...,aplikasi mobile banking ini cukup standar say...,"[{'label': 'negative', 'score': 0.502381682395...",negative,0.502382,aplikasi mobile banking standar transaksi dasa...,Opinion on Banking App,0.687159
